## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm.auto import tqdm

## Set Device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else device

## Create fully connected layer

In [4]:
class FullNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FullNN, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
model = FullNN(784, 10)
x = torch.randn(64, 784)

print(model(x).shape)

torch.Size([64, 10])


## Hyperparameters

In [6]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 20
grad_accum = 8

In [7]:
train_dataset = datasets.MNIST(root="data/", train=True, transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_dataset, train_dataloader

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data/
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 <torch.utils.data.dataloader.DataLoader at 0x110f48b20>)

In [8]:
test_dataset = datasets.MNIST(root="data/", train=False, transform=transforms.ToTensor(), download=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
test_dataset, test_dataloader

(Dataset MNIST
     Number of datapoints: 10000
     Root location: data/
     Split: Test
     StandardTransform
 Transform: ToTensor(),
 <torch.utils.data.dataloader.DataLoader at 0x110f48bb0>)

## Initialize Network

In [9]:
model = FullNN(input_size=input_size, num_classes=num_classes)
model = model.to(device)

## Loss and Optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

## Train the Model

In [11]:
for epoch in range(num_epochs):
    accum = 0
    optimizer.zero_grad()
    model.train()
    for batch in tqdm(train_dataloader, total=len(train_dataloader), desc=f"Epoch {epoch}"):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        x = x.view(x.size(0), -1)
        y_hat = model(x)

        loss = criterion(y_hat, y) / grad_accum
        loss.backward()

        accum += 1
        if accum % grad_accum == 0:
            optimizer.step()
            optimizer.zero_grad()

    model.eval()
    test_losses = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader, total=len(test_dataloader), desc=f"Epoch {epoch}"):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            x = x.view(x.size(0), -1)
            y_hat = model(x)
    
            loss = criterion(y_hat, y)
            test_losses += loss

    print(f"Epoch {epoch}: Test loss {test_losses/len(test_dataloader):.2f}")

Epoch 0:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0: Test loss 0.39


Epoch 1:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/157 [00:00<?, ?it/s]

...

Epoch 18:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 18: Test loss 0.10


Epoch 19:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 19:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 19: Test loss 0.10


In [15]:
def check_accuracy(loader, model):
    model.eval()
    cum_tp = 0
    cum_tot = 0
    with torch.no_grad():
        for batch in tqdm(loader, total=len(loader), desc=f"Testing"):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            x = x.view(x.size(0), -1)
            y_hat = model(x)

            preds = F.softmax(y_hat, dim=-1)
            _, pred_idx = preds.max(dim=-1)
            cum_tp += (pred_idx == y).sum()
            cum_tot += y.numel()

    print(f"Accuracy: {cum_tp/cum_tot*100:.4f}")

In [16]:
check_accuracy(train_dataloader, model)
check_accuracy(test_dataloader, model)

Testing:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 97.9850


Testing:   0%|          | 0/157 [00:00<?, ?it/s]

Accuracy: 96.9800
